In [1]:
import json
import pandas as pd
from kafka import KafkaConsumer
from dateutil import parser
import plotly.express as px

from datetime import datetime

consumer = KafkaConsumer(
    'darooghe.transaction_volume',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest', 
    enable_auto_commit=True,
    group_id='visualization-group-new',  
    consumer_timeout_ms=30000,  
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

data = []
max_messages = 50
print("Collecting transaction volume data...")

for i, msg in enumerate(consumer):
    value = msg.value
    print(f"[{i}] Raw Message:", value)

    try:
        window_start = parser.isoparse(value["window_start"])
        window_end = parser.isoparse(value["window_end"])
        transaction_count = int(value["transaction_count"])

        data.append({
            "window_start": window_start,
            "window_end": window_end,
            "transaction_count": transaction_count
        })
    except Exception as e:
        print(f"Error parsing message #{i}: {e}")
        continue

    if i >= max_messages:
        break

consumer.close()

if not data:
    print("No data received from Kafka topic.")
else:
    df = pd.DataFrame(data)
    df.sort_values(by="window_start", inplace=True)

    fig = px.line(
        df,
        x="window_start",
        y="transaction_count",
        title="Transaction Volume Over Time (1-Minute Windows)",
        labels={"window_start": "Time", "transaction_count": "Transaction Count"},
        markers=True
    )

    fig.update_layout(
        xaxis_title='Time',
        yaxis_title='Number of Transactions',
        template='plotly_dark'
    )
    fig.show()


[0] Raw Message: {'window_start': '2025-05-02T21:17:00.000+03:30', 'window_end': '2025-05-02T21:18:00.000+03:30', 'transaction_count': 855}
[1] Raw Message: {'window_start': '2025-05-02T21:17:00.000+03:30', 'window_end': '2025-05-02T21:18:00.000+03:30', 'transaction_count': 1824}
[2] Raw Message: {'window_start': '2025-05-02T21:17:00.000+03:30', 'window_end': '2025-05-02T21:18:00.000+03:30', 'transaction_count': 2008}
[3] Raw Message: {'window_start': '2025-05-02T21:18:00.000+03:30', 'window_end': '2025-05-02T21:19:00.000+03:30', 'transaction_count': 671}
[4] Raw Message: {'window_start': '2025-05-02T21:18:00.000+03:30', 'window_end': '2025-05-02T21:19:00.000+03:30', 'transaction_count': 1476}
[5] Raw Message: {'window_start': '2025-05-02T21:19:00.000+03:30', 'window_end': '2025-05-02T21:20:00.000+03:30', 'transaction_count': 127}
[6] Raw Message: {'window_start': '2025-05-02T21:18:00.000+03:30', 'window_end': '2025-05-02T21:19:00.000+03:30', 'transaction_count': 2049}
[7] Raw Message: